In [1]:
import numpy as np

import librosa
import librosa.display

import soundfile

import matplotlib.pyplot as plt

import os

# Process Monophonic samples

- remove noise at the beginning by onset detection
- normalise to -6dBFS

In [2]:
nofx_path = '/Volumes/STEVE/DATASETS/IDMT-SMT-AUDIO-EFFECTS/Gitarre monophon/Samples/NoFX'
out_path = '/Volumes/Macintosh HD/DATASETS/NoFX_mono_preprocessed'

# constants
_sr = 44100
_n_fft = 2048
_win_length = _n_fft
_hop_length = int(_win_length/4)

# fade in vector
_fade_length_in_sec = 0.02
_fade_length_in_samples = int(_fade_length_in_sec * _sr)
fade_amp = np.linspace(0, 1, num=_fade_length_in_samples, endpoint=True, retstep=False, dtype=None, axis=0)

In [3]:
if not os.path.exists('%s' % out_path):
    os.makedirs('%s' % out_path)

for file in os.listdir(nofx_path):
    if not(file.startswith("._")) and file.endswith(".wav"):
        filename = file[:-4]
        # open file
        audio_file, _ = librosa.load("%s/%s.wav" % (nofx_path, filename), 
                                     sr=_sr, 
                                     mono=True,
                                     offset=0.0, 
                                     duration=None, 
                                     dtype=np.float32, 
                                     res_type='kaiser_best')
        # clean file:
            # envelope
        oenv_raw = librosa.onset.onset_strength(y=audio_file, sr=_sr)
            # onset without backtrack
        onset_raw = librosa.onset.onset_detect(onset_envelope=oenv_raw, backtrack=False)
            # get main onset index
        main_onset_idx = np.argmax(oenv_raw[onset_raw])
            # backtrack onsets
        onset_raw_bt = librosa.onset.onset_backtrack(onset_raw, oenv_raw)
            # onset times
        ons_times_in_samples = librosa.frames_to_samples(onset_raw_bt, hop_length=_hop_length, n_fft=None)
            # apply fade in
        audio_file_proc = np.concatenate((
            audio_file[0:ons_times_in_samples[main_onset_idx]-_fade_length_in_samples] * 0,
            audio_file[ons_times_in_samples[main_onset_idx]-_fade_length_in_samples:ons_times_in_samples[main_onset_idx]] * fade_amp,
            audio_file[ons_times_in_samples[main_onset_idx]:]))
        # normalise to -6dBFS
        audio_file_proc = audio_file_proc / (2 * max(abs(audio_file_proc)))
        # write file
        soundfile.write(file="%s/%s.wav" % (out_path, filename), 
                        data=audio_file_proc, 
                        samplerate=_sr, 
                        subtype='PCM_16')
print('DONE!')

DONE!


# Process Polyponic samples

- normalise to -6dBFS

In [5]:
nofx_path = '/Volumes/STEVE/DATASETS/IDMT-SMT-AUDIO-EFFECTS/Gitarre polyphon/Samples/NoFX'
out_path = '/Volumes/Macintosh HD/DATASETS/NoFX_poly_preprocessed'

# constants
_sr = 44100
_n_fft = 2048
_win_length = _n_fft
_hop_length = int(_win_length/4)

In [6]:
if not os.path.exists('%s' % out_path):
    os.makedirs('%s' % out_path)

for file in os.listdir(nofx_path):
    if not(file.startswith("._")) and file.endswith(".wav"):
        filename = file[:-4]
        # open file
        audio_file, _ = librosa.load("%s/%s.wav" % (nofx_path, filename), 
                                     sr=_sr, 
                                     mono=True,
                                     offset=0.0, 
                                     duration=None, 
                                     dtype=np.float32, 
                                     res_type='kaiser_best')
        # normalise to -6dBFS
        audio_file_proc = audio_file / (2 * max(abs(audio_file)))
        # write file
        soundfile.write(file="%s/%s.wav" % (out_path, filename), 
                        data=audio_file_proc, 
                        samplerate=_sr, 
                        subtype='PCM_16')
print('DONE!')

DONE!
